# 1. 异常处理简明解释
try：包裹可能引发异常的代码块。

except：捕获并处理特定类型的异常。

else（可选）：当没有异常时执行。

finally（可选）：无论是否发生异常，最终必须执行的代码（如资源清理）。

用途：防止程序因未处理的异常崩溃，优雅处理错误场景。
常见场景：文件操作、网络请求、用户输入验证、数据库连接。

In [ ]:
# 2.代码示例

def read_file(file_path):
    try:
        file = open(file_path, "r")
        content = file.read()
        number = int(content)  # 尝试转换为整数
    except FileNotFoundError:
        print("文件不存在！")
    except ValueError:
        print("文件内容不是有效数字！")
    else:
        print(f"读取成功，数字是：{number}")
    finally:
        file.close()  # 确保文件始终关闭
        print("资源已清理")

# 测试
read_file("data.txt")  # 假设文件不存在
read_file("valid.txt")  # 假设文件内容为 "42"
read_file("invalid.txt") # 假设文件内容为 "hello"
""" 预期输出
文件不存在！
资源已清理
读取成功，数字是：42
资源已清理
文件内容不是有效数字！
资源已清理 """

# 3. 常见错误与避免方法

错误1：捕获所有异常但不处理
```python
try:
    # 某些操作
except:
    pass  # 静默吞掉所有异常！
```
问题：隐藏潜在错误，导致调试困难。
修复：明确捕获特定异常，记录错误信息。

错误2：except顺序错误
```python
try:
    # 某些操作
except Exception as e:  # 捕获所有异常
    print("通用错误")
except FileNotFoundError:  # 永远不会执行！
    print("文件不存在")
```

问题：父类异常在前会屏蔽子类异常。
修复：先捕获具体异常，再捕获通用异常。

错误3：在finally中返回
```python
def risky():
    try:
        return "成功"
    finally:
        return "覆盖结果"  # 会覆盖前面的返回值！
```

问题：finally中的return会覆盖try或except的返回值。
修复：避免在finally中使用return，仅做清理操作。

4. 实际应用场景
场景1：文件操作自动关闭资源

```python
try:
    file = open("data.txt", "r")
    process(file.read())
except FileNotFoundError:
    print("文件不存在")
finally:
    file.close()  # 确保文件句柄释放
```
场景2：网络请求重试机制

```python
import requests
retries = 3
for _ in range(retries):
    try:
        response = requests.get("https://api.example.com", timeout=5)
        response.raise_for_status()  # 检查HTTP错误
        break
    except (requests.Timeout, requests.HTTPError) as e:
        print(f"请求失败，重试中... ({_+1}/{retries})")
else:
    print("所有重试均失败！")
```


# 5. 练习题

初级
编写一个函数，处理用户输入的两个数字除法，捕获除零错误并提示。

中级
读取一个CSV文件，处理可能发生的文件不存在、字段缺失或数据格式错误（如非数字值）。

    高级题：实现一个日志分析器
题目描述
编写一个Python程序，分析一个日志文件（log.txt），统计以下信息：

总行数：日志文件的总行数。

错误日志数量：统计包含关键字 ERROR 的行数。

按日期统计日志数量：假设日志每行的开头是日期（格式为 YYYY-MM-DD），统计每个日期的日志数量。

捕获并处理异常：处理文件不存在、文件格式错误等异常。

日志文件示例（log.txt）：

复制
2023-08-20 12:00:00 INFO: 系统启动
2023-08-20 12:01:00 ERROR: 数据库连接失败
2023-08-21 09:15:00 INFO: 用户登录
2023-08-21 09:20:00 ERROR: 文件未找到
2023-08-21 10:00:00 INFO: 数据处理完成
输出示例：

复制
总行数: 5
错误日志数量: 2
按日期统计日志数量:
2023-08-20: 2
2023-08-21: 3
要求
使用 try-except 捕获文件不存在异常（FileNotFoundError）。

如果日志文件的某行格式不符合要求（如日期格式错误），跳过该行并记录错误日志。

使用字典统计每个日期的日志数量。

将统计结果保存到一个新的文件（log_summary.txt）中。

In [ ]:
def divide_numbers():
    try:
        num1 = input("请输入第一个数字: ")
        num2 = input("请输入第二个数字: ")
        num1 = float(num1)
        num2 = float(num2)
        result = num1 / num2
        print(f"{num1} 除以 {num2} 的结果是: {result}")
    except ZeroDivisionError:
        print("错误：除数不能为0！")
    except ValueError:
        print("错误：输入的不是有效的数字！")

divide_numbers()


In [5]:
import csv,os
def read_csv_file(path):
    requried_col=['月份','类别','销售额']
    if not os.path.exists(path):
        print(f"文件路径不存在")
        return
    with open(path,'r',encoding='utf-8-sig') as f:
        reader=csv.DictReader(f)
        print(set(reader.fieldnames))
        if set(requried_col)-set(reader.fieldnames):
            missing_col=set(requried_col)-set(reader.fieldnames)
            print(f"缺失必需列{missing_col}")
            return
        for row in reader:
            month=row['月份']
            category=row['类别']
            try:
                sales=float(row['销售额'])
                print(f"月份{month}，类别{category}，销售额{sales}")
            except ValueError:
                print(f"警告，在{reader.line_num}中，销售额列的值是无效数字已跳过")
read_csv_file("sales.csv")

{'类别', '月份', '销售额'}
月份2023-01，类别电子产品，销售额150000.0
月份2023-01，类别服饰，销售额80000.0
警告，在4中，销售额列的值是无效数字已跳过


In [1]:
from datetime import datetime

import logging
def ana_log_file(input_log,output_log):
    try:
        total_line=0
        total_error=0
        date_count = {}
        with open(input_log,'r',encoding='utf-8') as f :
            for line in f:
                total_line+=1
                if 'ERROR' in line:
                    total_error+=1
                try:
                    date_str=line.split()[0]
                    date=datetime.strptime(date_str,"%Y-%m-%d").date()#如果 date_str 是 "23-01-31"，那么执行这行代码后，date 将会是 datetime.date(2023, 1, 31)。
                    date_count[date]=date_count.get(date,0)+1#如果字典中不含key (date)则date_count.get(date,0)值为0
                except (IndexError,ValueError) as e:
                    logging.warning(f"跳过格式错误的行:{line.strip()}")
            with open(output_log, "w", encoding="utf-8") as output_file:
                output_file.write(f"总行数: {total_line}\n")
                output_file.write(f"错误日志数量: {total_error}\n")
                output_file.write("按日期统计日志数量:\n")
                for date, count in sorted(date_count.items()):
                    output_file.write(f"{date}: {count}\n")

            logging.info(f"日志分析完成，结果已保存至 {output_log}")
    except FileNotFoundError:
        logging.error(f"文件不存在: {input_log}")
    except Exception as e:
        logging.error(f"未知错误: {str(e)}")

ana_log_file("log.txt", "log_summary.txt")

# 思考问题
当你在多线程/协程环境中使用异常处理时，try-except块是否能捕获其他线程或协程中抛出的异常？如何设计一个健壮的异步错误处理机制？

请从以下角度思考：

    异常传播的范围
        在Python中，try-except块只能捕获当前线程或协程中抛出的异常。它 不能 捕获其他线程或协程中抛出的异常。这是因为每个线程和协程都有自己的执行流和栈，异常传播范围仅限于当前的执行上下文

线程/协程间错误的隔离与传递

全局异常钩子（如sys.excepthook）的局限性

结合日志系统记录跨线程错误
    为了更好地记录和调试多线程/协程环境中的异常，可以结合日志系统：
        
        -使用日志库 ：如logging库，可以在try-except块中捕获异常并记录到日志文件中。
        日志格式 ：记录异常的详细信息，包括异常类型、异常消息、堆栈跟踪等，以便后续分析